In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

import sys
import json
# Add the parent directory to the path
sys.path.append('..')

# Now you can import the config module
from config import groqkey, OPENAI_Key

In [8]:
# Define a function to replace "not numbered" with the value in "Variable Label"
def replace_not_numbered(row):
    if row["Question number\n (Questionnaire file)"] == "not numbered":
        return row["Variable Label\n (Data files)"]
    else:
        return row["Question number\n (Questionnaire file)"]

#Wave 1 - All data of NL
Survey_Responses = pd.read_csv("..\dataverse_files\SCALAR_Coastal_Longitudinal_Study_Wave_One_NL.csv").set_index("ID")

In [9]:
Survey_Overview  = pd.read_excel('..\Survey Questions Overview.xlsx', sheet_name='Wave 1', engine='openpyxl')
# Clean the survey questions dataframe to make it more usable for mapping
# Fill forward non-null ADICO Category values to apply them to all relevant rows, Specify the columns to forward fill excluding "Values" and "Value labels"
columns_to_ffill = [col for col in Survey_Overview.columns if col not in ["Values", "Value labels"]]

# Forward fill the specified columns
Survey_Overview[columns_to_ffill] = Survey_Overview[columns_to_ffill].ffill()
Survey_Overview = Survey_Overview.rename(columns={'ADICO Category': 'ADICO_Category'})

# Apply the function to replace "not numbered" with the value in "Variable Label"
Survey_Overview["Question number\n (Questionnaire file)"] = Survey_Overview.apply(replace_not_numbered, axis=1)
Survey_Overview['Value labels'] = Survey_Overview['Value labels'].astype(str)

# Set the index to {value of "Question number\n (Questionnaire file)"} + "_" + {str(value of "Values")}
Survey_Overview.set_index(Survey_Overview["Variable Label\n (Data files)"] + "_" + Survey_Overview["Values"].astype(str), inplace=True)
Survey_Overview.drop("ID_nan", inplace=True)
Survey_Overview.drop_duplicates(inplace=True)

question_answers_list = []
# Define a function to create the combined string
def combine_description_and_labels(group, question_answers_list, qnum):
    question_subset = Survey_Overview[Survey_Overview["Question number\n (Questionnaire file)"] == group["Question number\n (Questionnaire file)"].iloc[0]]
    # Check if it's the first row instance with the current "Question number\n (Questionnaire file)" column value
    first_instance_index = question_subset[question_subset.duplicated(subset=["Question number\n (Questionnaire file)"], keep="first")].index
    combined_string = ""
    # Concatenate the current row's "Description" and all "Value labels" values
    combined_string += str(group["Description"].iloc[0]) + ", " #+ "; ".join(group["Value labels"].astype(str))
    
    if first_instance_index.size != 0:
        first_description = question_subset["Description"].iloc[0]
        if first_description != group["Description"].iloc[0]:
         # If not the first instance, start with the first instance's "Description" column value
            combined_string += first_description

    question_answers_list = question_answers_list + [combined_string] * group.shape[0]  # Extend the list with the combined strings
    return question_answers_list

# Group by "Variable Label\n (Data files)" and apply the function to create the combined string
for group in Survey_Overview.groupby("Variable Label\n (Data files)",sort=False):
    question_answers_list = combine_description_and_labels(group[1], question_answers_list, group[1]["Question number\n (Questionnaire file)"].iloc[0])
Survey_Overview["question_answers_combined"] = question_answers_list


# Since the dataset has multiple rows per question for different value labels, we'll create a unique mapping
# Create the new mapping dictionary
question_adico_mapping = Survey_Overview[['question_answers_combined','ADICO_Category',"Variable Label\n (Data files)"]].drop_duplicates().set_index('question_answers_combined')

# Filter out questions that are categorized as Attributes, Conditions, or Aims for clarity in analysis
Attributes = list(set([Attribute for Attribute in question_adico_mapping[question_adico_mapping['ADICO_Category'].isin(["Attribute", "Attribute/Condition"])]['Variable Label\n (Data files)']]))
Conditions = list(set([Condition for Condition in question_adico_mapping[question_adico_mapping['ADICO_Category'].isin(["Condition",'Aim/Condition', "Attribute/Condition"])]['Variable Label\n (Data files)']]))
Aims = list(set([Aim for Aim in question_adico_mapping[question_adico_mapping['ADICO_Category'].isin(["Aim", "Aim/Condition"])]['Variable Label\n (Data files)']]))
Attcons = list(set([Attcon for Attcon in question_adico_mapping[question_adico_mapping['ADICO_Category'].isin(["Attribute", "Condition",'Aim/Condition', "Attribute/Condition"])]['Variable Label\n (Data files)']]))

c:\Users\coolg\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [10]:
#Selected attributes, aims, and conditions we would like to explore and test 
chosen_aims =  [
    "R2_implementation_SM1",
    "R2_implementation_SM2",
    "R2_implementation_SM3",
    "R2_implementation_SM6",
    "R2_implementation_NM4",
    "R2_implementation_NM2",
    "R2_implementation_NM3"
]
base_conditions = [
    # "R05_worry",
    # "Q0_age",
]
efficacy_conditions = [
    "R1a_self_efficacy_SM1",
    "R1a_self_efficacy_SM2",
    "R1a_self_efficacy_SM3",
    "R1a_self_efficacy_SM6",
    "R1a_self_efficacy_NM4",
    "R1a_self_efficacy_NM2",
    "R1a_self_efficacy_NM3",
    ]
cost_conditions = [
    "R1c_perc_cost_SM1",
    "R1c_perc_cost_SM2",
    "R1c_perc_cost_SM3",
    "R1c_perc_cost_SM6",
    "R1c_perc_cost_NM4",
    "R1c_perc_cost_NM2",
    "R1c_perc_cost_NM3",
]

chosen_attributes = []


chosen_conditions = base_conditions + efficacy_conditions + cost_conditions

adjusted_responses = Survey_Responses.copy().dropna(axis=1)

for column in adjusted_responses.columns:
    if "self_efficacy" in column or "perc_cost" in column or "worry" in column:
        #Preprocess key condition question responses to yes or no
        adjusted_responses.loc[adjusted_responses[column] > 3, column] = 5
        adjusted_responses.loc[adjusted_responses[column] <= 3, column] = 1
    if "implementation" in column:
        #Simplify the implementation responses to yes i intend to or no I not intend to make this change
        adjusted_responses.loc[adjusted_responses[column] > 5, column] = 6 #if they dont intend to
        # adjusted_responses.loc[adjusted_responses[column] == 1, column] = 7 #if they have already made the change
        adjusted_responses = adjusted_responses[adjusted_responses[column] != 1]
        adjusted_responses.loc[adjusted_responses[column] <= 5, column] = 2 #if they intend to

#exclude any "dont know" or "prefer not to say responses" from the question overview data
Survey_Overview = Survey_Overview[Survey_Overview['Values'] < 90]


# Assuming Survey_Overview is your DataFrame and 'Variable Label\n (Data files)' is the column you want to filter
chosen_aims_overview = Survey_Overview[Survey_Overview['Variable Label\n (Data files)'].isin(chosen_aims)][['question_answers_combined', 'Value labels']]
chosen_conditions_overview = Survey_Overview[Survey_Overview['Variable Label\n (Data files)'].isin(chosen_conditions)][['question_answers_combined', 'Value labels']]
chosen_attributes_overview = Survey_Overview[Survey_Overview['Variable Label\n (Data files)'].isin(chosen_attributes)][['question_answers_combined', 'Value labels']]

# Check if any of the listed columns doesn't exist in the repsonses data 
for column in chosen_conditions+chosen_aims:
    try:
        adjusted_responses[column]
    except: 
        print(column, "not in Survey responses")

In [11]:
import plotly.graph_objects as go

# Define a dictionary to map specific values to colors
color_dict = {2: '#00a6d6',
              6: '#a7a7a7'}

# Loop through each item in the list of chosen aims
for item in range(len(chosen_aims)):
    aim = chosen_aims[item]
    
    # Combine base conditions with efficacy and cost conditions for the current aim
    conditions = base_conditions + [efficacy_conditions[item]] + [cost_conditions[item]]
    
    # Get unique values for the current aim
    unique_aim_values = adjusted_responses[aim].unique()
    
    # Filter survey data to get the question data related to the current aim
    aim_question_data = Survey_Overview[Survey_Overview['Variable Label\n (Data files)'] == aim]

    aim_question_data.loc[aim_question_data['Values'] == 2, "Value labels"] = "I intend to"
    # Initialize lists to store values, colors, sources, targets, labels, and added sources
    values = []
    colors = []
    sources = []
    targets = []
    labels = []
    added_source_values = []
    node_labels = []
    
    # Initialize counters
    outcomes_added = 0
    attribute_count = 0
    
    # Loop through each unique value of the current aim to generate labels
    for aim_value in unique_aim_values:
        labels.append(aim_question_data[aim_question_data['Values'] == aim_value]["Value labels"].values[0])
        node_labels.append(f"{labels[-1]} ({adjusted_responses[adjusted_responses[aim] == aim_value].shape[0]})")
    
    # Loop through each condition to generate labels for each condition value
    for condition in conditions:
        condition_vals = sorted(adjusted_responses[condition].unique())
        condition_question_data = Survey_Overview[Survey_Overview['Variable Label\n (Data files)'] == condition]
        
        for condition_value in condition_vals:
            label = condition + ": " + condition_question_data[condition_question_data['Values'] == condition_value]["Value labels"].values[0]
            labels.append(label)
            node_labels.append(f"{label} ({adjusted_responses[adjusted_responses[condition] == condition_value].shape[0]})")
    
    # Loop through each unique value of the current aim to generate sources, targets, and values
    for aim_value in unique_aim_values:
        total_added = 0
        
        # Get the values of the first condition for the current aim value
        condition_vals = adjusted_responses[adjusted_responses[aim] == aim_value][conditions[0]]
        new_values = list(condition_vals.value_counts()[sorted(condition_vals.unique())])
        
        # Generate sources and values
        sources += [total_added + outcomes_added] * len(new_values)
        values += new_values
        total_added += len(new_values)
        colors += [color_dict[aim_value]] * len(new_values)
        
        # Generate targets
        targets += range(total_added - len(new_values) + len(unique_aim_values), total_added + len(unique_aim_values))
        
        # Initialize target and attribute counters
        target_count = attribute_count = len(new_values)
        pre_cond_count = 0
        source_count = 0
        
        # Loop through each subsequent condition to generate sources, targets, and values
        for condition in conditions[1:]:
            previous_condition_values = sorted(adjusted_responses[conditions[pre_cond_count]].unique())
            target_count += len(previous_condition_values)
            
            for previous_condition_value in previous_condition_values:
                aimfiltered = adjusted_responses[adjusted_responses[aim] == aim_value]
                conditionfiltered_vals = aimfiltered[aimfiltered[conditions[pre_cond_count]] == previous_condition_value][condition]
                
                new_values = list(conditionfiltered_vals.value_counts()[sorted(conditionfiltered_vals.unique())])
                
                sources += [attribute_count + source_count] * len(new_values)
                source_count += 1
                values += new_values
                total_added += len(new_values)
                colors += [color_dict[aim_value]] * len(new_values)
                targets += range(target_count, target_count + len(new_values))
            
            pre_cond_count += 1
        
        outcomes_added += 1
    df = pd.DataFrame
    # Create the Sankey diagram using Plotly
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=node_labels,
            color="blue"
        ),
        link=dict(
            source=sources,  # indices correspond to labels
            target=targets,
            label=values,
            value=values,
            color=colors
        )
    ,)])
    
    # Update the layout of the figure with a title and font size
    fig.update_layout(title_text="People living in the Netherlands: " + aim_question_data[aim_question_data['Values'] == aim_value]["Description"].iloc[0])
    

    # Display the figure
    fig.show()
